# Functional Programming for Data Analysis

### Jim Pivarski

Second notebook: functional playground

C++ and Python are not functional languages.

Functional programming is a nebulously defined style, so there isn't a strict definition, but generally it involves working with expressions and not statements.

   * **Expression:** tree-like structure of nested function calls. Has a return value and can be used as an argument to a function. Examples: a FORTRAN formula, a diagrammed sentence, all of Lisp.
   * **Statement:** a command that either changes the computer's state or does nothing. Examples: Python's `for` and `if`, `move-robotic-arm`, all of assembly language.

This notebook will add methods to Python lists to make them easier to use for functional programming.

The goal will be to analyze data without ever writing a `for` loop or `if` statement.

In [ ]:
%matplotlib inline
import helpers.functional

In [ ]:
[1, 2, 3, 4, 5].map(lambda x: x**2)

In [ ]:
[1, 2, 3, 4, 5].map(lambda x: x**2).filter(lambda x: x % 2 == 0)

In [ ]:
[1, 2, 3, 4, 5].map(lambda x: x**2).filter(lambda x: x % 2 == 0).reduce(lambda x, y: x + y)

In [ ]:
from helpers.functional import events